# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of a *C++* library.

First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

On Windows OSes, the visual studio version used to compile future wrappers must be given.

In [2]:
if is_windows:
    kwargs = dict(msvc_version = '12.0')
else:
    kwargs = dict()

* to detect the version of *Python* installed and to save it in the `PYTHON_VERSION` environment variable.

In [3]:
import sys
PYTHON_VERSION = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

* to import **AutoWIG**.

In [4]:
import autowig

* to import **subprocess**.

In [5]:
import subprocess

* to detect the **Git** repository root

In [6]:
import os
GIT_ROOT = subprocess.check_output('git rev-parse --show-toplevel', shell=True).decode()
GIT_ROOT = GIT_ROOT.replace('/', os.sep).strip()

Then, we need to install and compile the *C++* library.
To do so, we use available **Conda** recipes.

In [7]:
subprocess.call('conda remove -y libbasic', shell=True)
CONDA_RECIPE = os.path.join(GIT_ROOT, 'bin', 'conda', 'libbasic')
subprocess.check_call('conda build --python=' + PYTHON_VERSION + ' ' + CONDA_RECIPE + ' -c statiskit -c defaults --override-channels',
                      shell=True)
subprocess.check_call('conda install -y libbasic --use-local -c statiskit -c defaults --override-channels',
                      shell=True)

0

Once these preliminaries are done, we can proceed to the actual generation of wrappers for this *C++* library.
For this, we create an empty Abstract Semantic Graph (ASG).

In [8]:
asg = autowig.AbstractSemanticGraph()

We then parse the headers of this *C++* library with relevant compilation flags.

In [9]:
%%time

try:
    from path import path as Path
except:
    from path import Path
prefix = Path(sys.prefix).abspath()
if is_windows:
    headers = [prefix/'Library'/'include'/'basic'/'overload.h',
               prefix/'Library'/'include'/'basic'/'binomial.h']
else:
    headers = [prefix/'include'/'basic'/'overload.h',
               prefix/'include'/'basic'/'binomial.h']
    
flags = ['-x', 'c++', '-std=c++11']
if is_windows:
    flags.append('-I' + str((prefix/'Library'/'include').abspath()))
else:
    flags.append('-I' + str((prefix/'include').abspath()))
    
if autowig.parser.plugin == 'libclang':
    kwargs['silent'] = True
    
asg = autowig.parser(asg, headers,
                          flags,
                          **kwargs)

CPU times: user 440 ms, sys: 20 ms, total: 460 ms
Wall time: 550 ms


Since most of **AutoWIG** guidelines are respected in this *C++* library, the `default` `controller` implementation is suitable.

In [10]:
%%time

asg = autowig.controller(asg)

CPU times: user 244 ms, sys: 4 ms, total: 248 ms
Wall time: 249 ms


In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [11]:
%%time

autowig.generator.plugin = 'boost_python_internal'  
wrappers = autowig.generator(asg,
                             module = os.path.join(GIT_ROOT, 'src', 'py', '_basic.cpp'),
                             decorator = os.path.join(GIT_ROOT, 'src', 'py', 'basic', '_basic.py'),
                             prefix = 'wrapper_')

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 48.1 ms


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [12]:
%%time

wrappers.write()

CPU times: user 412 ms, sys: 524 ms, total: 936 ms
Wall time: 2.7 s


Here is the list of the generated wrappers (untracked files).

In [13]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../../src/py/_basic.cpp
	../../src/py/_basic.h
	../../src/py/basic/_basic.py
	../../src/py/wrapper_0f744e8d056f5d469a887c7c78eaf8fe.cpp
	../../src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp
	../../src/py/wrapper_92d464d1243554009adc48a065a5be3e.cpp
	../../src/py/wrapper_f926cb231a7f5da09f313cd361ff94c7.cpp

nothing added to commit but untracked files present (use "git add" to track)


And here, we present the wrappers generated for the `BinomialDistribution` class.

In [14]:
WRAPPER = os.path.join(GIT_ROOT, 'src', 'py', 'wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp')
!pygmentize {WRAPPER}

#include "_basic.h"



namespace autowig
{

}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::BinomialDistribution const volatile * get_pointer<class ::BinomialDistribution const volatile >(class ::BinomialDistribution const volatile *c) { return c; }
}
    #endif
#endif



void wrapper_4046a8421fe9587c9dfbc97778162c7d()
{

    double  (::BinomialDistribution::*method_pointer_3a3ff64f25e358a6a10b1cd3b3425b82)(unsigned int const) const = &::BinomialDistribution::pmf;
    double  (::BinomialDistribution::*method_pointer_3c97a500c9575c259d5cbdd76120ff4f)() const = &::BinomialDistribution::get_pi;
    void  (::BinomialDistribution::*method_pointer_d364a0529e33516f8ecbb7dcedd60aa0)(double const) = &::BinomialDistribution::set_pi;
    boost::python::class_< class ::BinomialDistribution, autowig::Held< class ::BinomialDistribution >::Type > class_4046a8421fe9587c9dfbc97778162c7d("BinomialDistribution", "", boost::python::no_init);
    class_4046a842

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [15]:
subprocess.call('conda remove -y python-basic', shell=True)
CONDA_RECIPE = os.path.join(GIT_ROOT, 'bin', 'conda', 'python-basic')
subprocess.check_call('conda build --python=' + PYTHON_VERSION + ' ' + CONDA_RECIPE + ' -c statiskit -c defaults --override-channels',
                      shell=True)
subprocess.check_call('conda install -y python-basic --use-local -c statiskit -c defaults --override-channels',
                      shell=True)

0

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [16]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [17]:
binomial.pmf(0)

0.5

In [18]:
binomial.pmf(1)

0.5

In [19]:
binomial.n = 0
binomial

In [20]:
binomial.pmf(0)

1.0

In [21]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print(error)
else:
    raise Exception('A `basic.ProbabilityError` should have been raised')

a probability must be in the interval [0,1]


Here is a report concerning objects wrapped using this notebook.

In [22]:
import fp17
fp17.report(asg)

Headers: 3 (170 SLOC)
Variables: 1 (1 wrapped)
Functions: 15 (10 wrapped)
Classes: 4 (4 wrapped)
